In [1]:
# 1) Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ajuster les options d'affichage de Pandas pour les tableaux
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

# Téléchargement des ressources NLTK nécessaires (stopwords, punkt, punkt_tab)
try:
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True) # Spécifique pour la tokenisation en français
    print("Ressources NLTK vérifiées/téléchargées.")
except Exception as e:
    print(f"Erreur lors du téléchargement des ressources NLTK : {e}")
    print("Veuillez vous assurer que 'nltk' est installé (`pip install nltk`) et que les ressources sont téléchargées manuellement.")

print("Bibliothèques importées et configuration initiale terminée.")

Ressources NLTK vérifiées/téléchargées.
Bibliothèques importées et configuration initiale terminée.


In [2]:
#2) Chargement des données
try:
    jobs_df = pd.read_csv('merged_jobs_final.csv')
    universities_df = pd.read_csv('university4.csv')
    print("Données chargées avec succès.")
except FileNotFoundError:
    print("Erreur: Fichiers CSV non trouvés. Assurez-vous que 'merged_jobs_final.csv' et 'university4.csv' sont dans le même dossier.")
    exit()

Données chargées avec succès.


In [3]:
#3) Préparation des données: Combinaison des colonnes textuelles pertinentes
# Pour les offres d'emploi (jobs_df)
jobs_df['combined_text'] = jobs_df['title'].fillna('') + ' ' + \
                          jobs_df['description'].fillna('') + ' ' + \
                          jobs_df['requirements'].fillna('') + ' ' + \
                          jobs_df['Competence'].fillna('')

# Pour les universités/formations (universities_df)
universities_df['combined_text'] = universities_df["Nom de l'Université/École"].fillna('') + ' ' + \
                                   universities_df['Nom du Programme/Diplôme'].fillna('') + ' ' + \
                                   universities_df['Compétences Acquises/Enseignées'].fillna('') + ' ' + \
                                   universities_df['Domaine(s) IT Couvert(s)'].fillna('')

print("Colonnes textuelles combinées pour les jobs et les universités.")
# Préparation des stopwords (mots courants à ignorer)
stop_words_fr = set(stopwords.words('french'))
stop_words_en = set(stopwords.words('english'))
all_stop_words = stop_words_fr.union(stop_words_en)

# Fonction de prétraitement du texte: Nettoyage et tokenisation
def preprocess_text(text):
    if pd.isna(text) or not isinstance(text, str): return ""
    text = text.lower()  # Mise en minuscule
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Suppression de la ponctuation et des chiffres
    tokens = word_tokenize(text, language='french') # Tokenisation en mots
    # Suppression des stopwords et des mots très courts
    tokens = [word for word in tokens if word not in all_stop_words and len(word) > 1]
    return ' '.join(tokens)

# 4) Nouvelle fonction specifie pour nettoyer des compétences sous forme d'ensemble
def clean_and_split_skills(skills_string):
    if pd.isna(skills_string) or not isinstance(skills_string, str):
        return set()
    # Mettre en minuscule, remplacer les virgules et autres séparateurs par des espaces
    cleaned_string = re.sub(r'[,\-/\(\)]', ' ', skills_string.lower())
    # Supprimer la ponctuation restante et les chiffres, puis diviser en mots
    tokens = re.findall(r'[a-zA-Z]+', cleaned_string)
    # Filtrer les stopwords et les mots courts
    filtered_tokens = [word for word in tokens if word not in all_stop_words and len(word) > 1]
    return set(filtered_tokens)


# Application du prétraitement aux DataFrames
jobs_df['processed_text'] = jobs_df['combined_text'].apply(preprocess_text)
universities_df['processed_text'] = universities_df['combined_text'].apply(preprocess_text)
print("Textes prétraités pour l'analyse des similarités.")

Colonnes textuelles combinées pour les jobs et les universités.
Textes prétraités pour l'analyse des similarités.


In [4]:
#5) Initialisation du vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.8)

# Ajuster le vectoriseur sur tous les textes (jobs et universités) pour un vocabulaire unifié
all_texts = pd.concat([jobs_df['processed_text'], universities_df['processed_text']], axis=0)
tfidf_vectorizer.fit(all_texts)

# Transformation des textes en matrices TF-IDF numériques
job_tfidf_matrix = tfidf_vectorizer.transform(jobs_df['processed_text'])
university_tfidf_matrix = tfidf_vectorizer.transform(universities_df['processed_text'])
print(f"Textes convertis en vecteurs TF-IDF. Forme de la matrice jobs: {job_tfidf_matrix.shape}, universités: {university_tfidf_matrix.shape}.")

Textes convertis en vecteurs TF-IDF. Forme de la matrice jobs: (3003, 2325), universités: (188, 2325).


In [5]:
#6) Fonction pour recommander des universités/formations avec compétences manquantes
def recommend_universities(job_title, job_description, job_requirements, job_skills_raw, top_n=10):
    # Prétraitement et vectorisation du job pour la similarité
    job_text = f"{job_title} {job_description} {job_requirements} {job_skills_raw}"
    processed_job_text = preprocess_text(job_text)
    job_vector = tfidf_vectorizer.transform([processed_job_text])

    # Extraction des compétences du job sous forme d'ensemble
    job_skills_set = clean_and_split_skills(job_skills_raw)

    # Calcul de la similarité cosinus avec toutes les formations
    similarity_scores = cosine_similarity(job_vector, university_tfidf_matrix).flatten()

    # Ajout des scores de similarité au DataFrame des universités
    universities_df['similarity_score'] = similarity_scores

    #7)  Tri des formations par score de similarité décroissant
    recommendations = universities_df.sort_values(by='similarity_score', ascending=False).head(top_n).copy() # Utilisez .copy() pour éviter SettingWithCopyWarning

    # Calcul des compétences manquantes pour chaque recommandation
    missing_skills_list = []
    for index, row in recommendations.iterrows():
        uni_skills_raw = row['Compétences Acquises/Enseignées']
        uni_skills_set = clean_and_split_skills(uni_skills_raw)
        
        # Compétences demandées par le job mais non couvertes par la formation
        missing_skills = job_skills_set - uni_skills_set
        
        if missing_skills:
            missing_skills_list.append(", ".join(sorted(list(missing_skills))))
        else:
            missing_skills_list.append("Aucune (couverture complète)")
            
    recommendations['Compétences Manquantes'] = missing_skills_list

    # Sélection des colonnes pertinentes pour l'affichage des résultats
    display_columns = ["Nom de l'Université/École", 'Nom du Programme/Diplôme',
                       'Compétences Acquises/Enseignées', 'Domaine(s) IT Couvert(s)',
                       'similarity_score', 'Compétences Manquantes']
    final_recommendations = recommendations[display_columns]

    return final_recommendations

print("Fonction de recommandation 'recommend_universities' améliorée définie.")

Fonction de recommandation 'recommend_universities' améliorée définie.


In [6]:
#8)
from IPython.display import display 
# Définition de l'exemple de job pour lequel nous voulons des recommandations
example_job_title = "Développeur Full Stack Senior"
example_job_description = "Nous recherchons un développeur Full Stack expérimenté pour rejoindre notre équipe dynamique. Vous serez responsable de la conception, du développement et du déploiement d'applications web robustes."
example_job_requirements = "Minimum 5 ans d'expérience. Maîtrise des frameworks JavaScript (React, Angular, Vue.js), Node.js, Python/Django, bases de données SQL/NoSQL."
example_job_skills = "JavaScript, Python, Node.js, React, Angular, Vue.js, Django, SQL, NoSQL, Git, Docker, CI/CD, Agile, communication, résolution de problèmes, cloud, aws, azure"

# --- Affichage des Détails de l'Offre d'Emploi en Entrée ---
print("\n--- DÉTAILS DE L'OFFRE D'EMPLOI EN ENTRÉE ---")
input_job_data = {
    "Titre du poste": [example_job_title],
    "Description du poste": [example_job_description],
    "Exigences": [example_job_requirements],
    "Compétences clés": [example_job_skills]
}
input_job_df = pd.DataFrame(input_job_data)
#9)  Utilise display() pour un rendu HTML de la table avec bordures
display(input_job_df) 

# Obtention des recommandations
# Assurez-vous que les fonctions recommend_universities et preprocess_text des cellules précédentes sont définies et exécutées.
top_recommendations = recommend_universities(
    example_job_title,
    example_job_description,
    example_job_requirements,
    example_job_skills,
    top_n=10
)

# --- Affichage des Recommandations d'Universités et de Formations ---
print("\n--- RECOMMANDATIONS D'UNIVERSITÉS ET DE FORMATIONS (SORTIE) ---")
# Utilise display() pour un rendu HTML de la table avec bordures
display(top_recommendations) 

print("\n--- CONSEIL POUR L'UTILISATEUR ---")
print("Pour chaque formation recommandée, vérifiez la colonne 'Compétences Manquantes'.")
print("Ces compétences sont demandées par le poste mais pourraient ne pas être entièrement couvertes par la formation.")
print("Nous vous conseillons de faire des recherches ou de suivre des auto-formations pour acquérir ces compétences et ainsi compléter votre profil pour le poste visé.")
print("Le 'similarity_score' indique à quel point la formation est proche des compétences du poste (plus il est proche de 1.0, meilleure est la correspondance).")


--- DÉTAILS DE L'OFFRE D'EMPLOI EN ENTRÉE ---


,Titre du poste,Description du poste,Exigences,Compétences clés
0,Développeur Full Stack Senior,"Nous recherchons un développeur Full Stack expérimenté pour rejoindre notre équipe dynamique. Vous serez responsable de la conception, du développement et du déploiement d'applications web robustes.","Minimum 5 ans d'expérience. Maîtrise des frameworks JavaScript (React, Angular, Vue.js), Node.js, Python/Django, bases de données SQL/NoSQL.","JavaScript, Python, Node.js, React, Angular, Vue.js, Django, SQL, NoSQL, Git, Docker, CI/CD, Agile, communication, résolution de problèmes, cloud, aws, azure"



--- RECOMMANDATIONS D'UNIVERSITÉS ET DE FORMATIONS (SORTIE) ---


,Nom de l'Université/École,Nom du Programme/Diplôme,Compétences Acquises/Enseignées,Domaine(s) IT Couvert(s),similarity_score,Compétences Manquantes
97,ITSUP Casablanca,Licence Professionnelle en Développement WEB Full Stack et DevOps,"Conception et gestion des bases de données, Création de sites web, Méthodes DevOps et systèmes, Ingénierie logicielle, Méthodes et automatisation des tests, Management de projets agiles, Architecture et processus d'entreprise, Sécurité informatique, Gestion de bases de données, Audit réseau et système, Conception et maintenance des systèmes d'information (SI), Développement avec des technologies avancées, Personnalisation d'applications distribuées, Analyse et conception logicielle, Gestion de projets technologiques, Support utilisateur, Développement d'applications web complètes (frontend et backend), Frameworks et bibliothèques modernes (React, Angular, Node.js), Bases de données relationnelles et NoSQL, Outils de collaboration et d'intégration continue (Docker, Kubernetes, Git), Automatisation des processus de développement et de déploiement, Gestion des infrastructures Cloud, Création d'interfaces utilisateur interactives et responsives, Intégration des tests automatisés et de la surveillance des performances","Génie Logiciel & Développement, DevOps & Cloud",0.307985,"agile, aws, azure, cd, ci, communication, django, javascript, probl, python, solution, sql, vue"
107,Euromed Polytechnic School (Université Euromed de Fès),Ingénieur d'État en Full Stack Engineering and Multimedia,"Langage et communication, Gestion de projet, Réseaux informatiques, Technologies Web, Ingénierie des bases de données, Algorithmes Python avancés, Sécurité Full Stack, Conception d'expérience utilisateur et d'interface, Ingénierie logicielle, Gestion de bases de données, Frameworks PHP, Optimisation combinatoire et approches de solution, Infographie, Développement Front-end avancé, Plateforme Node.js, Cloud Computing, Git et GitHub, Environnement Java EE, Réalité virtuelle, Développement Back-end avancé, Administration Oracle, Technologies Blockchain, Développement mobile iOS/Android, Intégration de la réalité virtuelle, Introduction à l'intelligence artificielle, Développement mobile avancé, Orchestration et conteneurisation de services Web, DevOps Web et mobile, Machine learning pour les développeurs web, Pipelines CI/CD, Droit du travail et gestion des ressources humaines","Génie Logiciel & Développement, Data Science & IA, Cybersécurité, DevOps & Cloud",0.288036,"agile, angular, aws, azure, django, docker, javascript, nosql, probl, react, sql, vue"
3,EST Kénitra (Université Ibn Tofaïl),LUS – Ingénierie Informatique en Infrastructure Digitale et Développement Full Stack,"Développement d'applications web et mobile, Bases de données avancées, JavaScript, NodeJS, POO en Java, Modélisation avec UML et IFML, Méthodologie Web 2.0, Systèmes de gestion de contenu (CMS), Développement mobile natif et hybride, Développement Web Full stack, Administration du système Linux, Virtualisation, Développement JavaEE, Excel avancé","Génie Logiciel & Développement, Réseaux & Administration Systèmes, DevOps & Cloud",0.285667,"agile, angular, aws, azure, cd, ci, cloud, communication, django, docker, git, js, node, nosql, probl, python, react, solution, sql, vue"
4,EST Kénitra (Université Ibn Tofaïl),LUS – Ingénierie Web Full Stack,"Développement Web en PHP et Modèle MVC, Systèmes de Gestion de Contenu (CMS), Algorithmique et Programmation en Python, Bases de Programmation et de Développement Web, Programmation en JavaScript, Bases de Données, Langage de Modélisation Unifié (UML), Frameworks Back end, Sécurité des Applications Web, Outils de Conteneurisation et de Déploiement (DEVOPS)","Génie Logiciel & Développement, DevOps & Cloud",0.232954,"agile, angular, aws, azure, cd, ci, cloud, communication, django, docker, git, js, node, nosql, probl, react, solution, sql, vue"
9,CMC Rabat-Salé-Kénitra,Développement Digital: WEB FULL STACK,Dév


--- CONSEIL POUR L'UTILISATEUR ---
Pour chaque formation recommandée, vérifiez la colonne 'Compétences Manquantes'.
Ces compétences sont demandées par le poste mais pourraient ne pas être entièrement couvertes par la formation.
Nous vous conseillons de faire des recherches ou de suivre des auto-formations pour acquérir ces compétences et ainsi compléter votre profil pour le poste visé.
Le 'similarity_score' indique à quel point la formation est proche des compétences du poste (plus il est proche de 1.0, meilleure est la correspondance).
